In [20]:
import glob
import warnings
import numpy as np 
import pandas as pd
import plotly as py
import seaborn as sns
import statistics as stat
import plotly.express as px
import plotly.graph_objs as go
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None)
from plotly.offline import init_notebook_mode
init_notebook_mode(connected = True)
import matplotlib.pyplot as plt
%matplotlib inline

## DISTRICT DATA

In [21]:
district = pd.read_csv('../data/districts_info.csv')
district.head()

,district_id,state,locale,pct_black/hispanic,pct_free/reduced,county_connections_ratio,pp_total_raw
0,8815,Illinois,Suburb,"[0, 0.2[","[0, 0.2[","[0.18, 1[","[14000, 16000["
1,2685,NaN,NaN,NaN,NaN,NaN,NaN
2,4921,Utah,Suburb,"[0, 0.2[","[0.2, 0.4[","[0.18, 1[","[6000, 8000["
3,3188,NaN,NaN,NaN,NaN,NaN,NaN
4,2238,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
district.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 233 entries, 0 to 232
Data columns (total 7 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   district_id               233 non-null    int64 
 1   state                     176 non-null    object
 2   locale                    176 non-null    object
 3   pct_black/hispanic        176 non-null    object
 4   pct_free/reduced          148 non-null    object
 5   county_connections_ratio  162 non-null    object
 6   pp_total_raw              118 non-null    object
dtypes: int64(1), object(6)
memory usage: 12.9+ KB


In [23]:
district.shape

(233, 7)

In [24]:
# how many missing values exist or better still what is the % of missing values in the dataset?
def percent_missing(df):

    # Calculate total number of cells in dataframe
    totalCells = np.product(df.shape)

    # Count number of missing values per column
    missingCount = df.isnull().sum()

    # Calculate total number of missing values
    totalMissing = missingCount.sum()

    # Calculate percentage of missing values
    print("The district dataset contains", round(((totalMissing/totalCells) * 100), 2), "%", "missing values.")

percent_missing(district)

The district dataset contains 27.1 % missing values.


In [25]:
# missing values in every columns 
district.isna().sum()

district_id                   0
state                        57
locale                       57
pct_black/hispanic           57
pct_free/reduced             85
county_connections_ratio     71
pp_total_raw                115
dtype: int64

dropping the states rows hat dont have any states information 
reason is because tehy dont really have any info also in the eng dataset

In [26]:
#driping nan states 
district = district[district.state.notna()].reset_index(drop=True)


In [27]:
district.isna().sum()

district_id                  0
state                        0
locale                       0
pct_black/hispanic           0
pct_free/reduced            28
county_connections_ratio    14
pp_total_raw                58
dtype: int64

In [28]:
# fill missing with ffill method for columns (pct_free/reduced , pp_total_raw )

def fix_missing_ffill(df, col):
    df[col] = df[col].fillna(method='ffill')
    return df[col]


def fix_missing_bfill(df, col):
    df[col] = df[col].fillna(method='bfill')
    return df[col]

district['pct_free/reduced'] = fix_missing_ffill(district, 'pct_free/reduced')
district['county_connections_ratio'] = fix_missing_ffill(district, 'county_connections_ratio')
district['pp_total_raw'] = fix_missing_ffill(district, 'pp_total_raw')

# fill 'pp_total_raw ' column with mode 
#district['pp_total_raw'] = district['pp_total_raw'].fillna(district['pp_total_raw'].mode()[0])


In [29]:
district.isna().sum()

district_id                 0
state                       0
locale                      0
pct_black/hispanic          0
pct_free/reduced            0
county_connections_ratio    0
pp_total_raw                0
dtype: int64

the country ratio...all is the same so i suggest we remove it 
also i have tried to rmove the invervals and have the data as one .


In [30]:
for i in ['pct_black/hispanic', 'pct_free/reduced']:
    district[i] = district[i].apply(lambda x: float(x.split(',')[0][1:]) + 0.1)

district['pp_total_raw'] = district['pp_total_raw'].apply(lambda x: int(x.split(',')[0][1:]) + 1000)

district.drop('county_connections_ratio', axis = 1, inplace = True)

district.head(10)

,district_id,state,locale,pct_black/hispanic,pct_free/reduced,pp_total_raw
0,8815,Illinois,Suburb,0.1,0.1,15000
1,4921,Utah,Suburb,0.1,0.3,7000
2,5987,Wisconsin,Suburb,0.1,0.1,11000
3,3710,Utah,Suburb,0.1,0.5,7000
4,7177,North Carolina,Suburb,0.3,0.3,9000
5,9812,Utah,Suburb,0.1,0.3,7000
6,6584,North Carolina,Rural,0.5,0.7,9000
7,1044,Missouri,Suburb,0.1,0.1,11000
8,7457,Washington,City,0.5,0.5,13000
9,1904,Connecticut,Rural,0.1,0.1,13000


In [36]:
district 

,district_id,state,locale,pct_black/hispanic,pct_free/reduced,pp_total_raw
0,8815,Illinois,Suburb,0.1,0.1,15000
1,4921,Utah,Suburb,0.1,0.3,7000
2,5987,Wisconsin,Suburb,0.1,0.1,11000
3,3710,Utah,Suburb,0.1,0.5,7000
4,7177,North Carolina,Suburb,0.3,0.3,9000
...,...,...,...,...,...,...
171,9515,New York,Rural,0.1,0.5,19000
172,8103,Tennessee,Rural,0.3,0.5,9000
173,4929,Virginia,Rural,0.1,0.5,13000
174,7975,California,City,0.7,0.7,13000


In [38]:
district.isna().sum()

district_id           0
state                 0
locale                0
pct_black/hispanic    0
pct_free/reduced      0
pp_total_raw          0
dtype: int64

## Engagement 

In [58]:
PATH = '../data/engagement_data' 

temp = []

for dist in district.district_id.unique():
    df = pd.read_csv(f'{PATH}/{dist}.csv', index_col=None, header=0)
    df["district_id"] = dist
    temp.append(df)
    
    
engagement = pd.concat(temp)
engagement = engagement.reset_index(drop=True)
engagement.head(5)

,time,lp_id,pct_access,engagement_index,district_id
0,2020-01-27,32213.0,100.00,3000.00,8815
1,2020-02-25,90153.0,33.33,2666.67,8815
2,2020-02-25,99916.0,0.00,NaN,8815
3,2020-02-25,28504.0,0.00,NaN,8815
4,2020-02-25,95731.0,33.33,333.33,8815


In [59]:
engagement.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17435744 entries, 0 to 17435743
Data columns (total 5 columns):
 #   Column            Dtype  
---  ------            -----  
 0   time              object 
 1   lp_id             float64
 2   pct_access        float64
 3   engagement_index  float64
 4   district_id       int64  
dtypes: float64(3), int64(1), object(1)
memory usage: 665.1+ MB


In [60]:
engagement.isna().sum()

time                      0
lp_id                   399
pct_access             7362
engagement_index    4297347
district_id               0
dtype: int64

In [ ]:
##how should we handle the empty values in engagement

In [61]:
engagement['time'] = pd.to_datetime(engagement['time'])
        
    # Add time features

    # Holidays
engagement["holiday"] = np.where(engagement["time"].isin(["2020-01-30", "2020-01-31"]), 1,
        np.where((engagement["time"]>="2020-04-05") & (engagement["time"]<="2020-04-09"), 1,
        np.where((engagement["time"]>="2020-06-12") & (engagement["time"]<="2020-09-10"), 1,
        np.where((engagement["time"]>="2020-11-25") & (engagement["time"]<="2020-11-29"), 1,
        np.where((engagement["time"]>="2020-12-21") & (engagement["time"]<="2020-12-31"), 1, 0)))))

    # Before/After Outbreak
engagement["outbreak"] = np.where(engagement["time"]<="2020-03-24", 0, 1)


In [62]:
engagement['month']= pd.to_datetime(engagement['time']).dt.month_name()
engagement['weekday']= pd.to_datetime(engagement['time']).dt.day_name()

In [63]:
#we can make the month and weekday into numeric if thats agreed on 
engagement

,time,lp_id,pct_access,engagement_index,district_id,holiday,outbreak,month,weekday
0,2020-01-27,32213.0,100.00,3000.00,8815,0,0,January,Monday
1,2020-02-25,90153.0,33.33,2666.67,8815,0,0,February,Tuesday
2,2020-02-25,99916.0,0.00,NaN,8815,0,0,February,Tuesday
3,2020-02-25,28504.0,0.00,NaN,8815,0,0,February,Tuesday
4,2020-02-25,95731.0,33.33,333.33,8815,0,0,February,Tuesday
...,...,...,...,...,...,...,...,...,...
17435739,2020-12-31,43009.0,0.02,8.14,7164,1,1,December,Thursday
17435740,2020-12-31,62984.0,0.00,0.04,7164,1,1,December,Thursday
17435741,2020-12-31,90014.0,0.00,0.04,7164,1,1,December,Thursday
17435742,2020-12-31,61945.0,0.29,16.06,7164,1,1,December,Thursday


In [64]:
#we can make engagement into a csv

## products 

In [65]:
product_info = pd.read_csv('../data/products_info.csv')
product_info.head()

,LP ID,URL,Product Name,Provider/Company Name,Sector(s),Primary Essential Function
0,13117,https://www.splashmath.com,SplashLearn,StudyPad Inc.,PreK-12,LC - Digital Learning Platforms
1,66933,https://abcmouse.com,ABCmouse.com,"Age of Learning, Inc",PreK-12,LC - Digital Learning Platforms
2,50479,https://www.abcya.com,ABCya!,"ABCya.com, LLC",PreK-12,"LC - Sites, Resources & Reference - Games & Si..."
3,92993,http://www.aleks.com/,ALEKS,McGraw-Hill PreK-12,PreK-12; Higher Ed,LC - Digital Learning Platforms
4,73104,https://www.achieve3000.com/,Achieve3000,Achieve3000,PreK-12,LC - Digital Learning Platforms


In [66]:
product_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 372 entries, 0 to 371
Data columns (total 6 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   LP ID                       372 non-null    int64 
 1   URL                         372 non-null    object
 2   Product Name                372 non-null    object
 3   Provider/Company Name       371 non-null    object
 4   Sector(s)                   352 non-null    object
 5   Primary Essential Function  352 non-null    object
dtypes: int64(1), object(5)
memory usage: 17.6+ KB


In [67]:
product_info.isna().sum()

LP ID                          0
URL                            0
Product Name                   0
Provider/Company Name          1
Sector(s)                     20
Primary Essential Function    20
dtype: int64

splitting the primary essential columns

In [68]:
# we can use fowward fil of back file to fill the 20 missing values in sector , primary function main and sub function 
product_info['Sector(s)'] = fix_missing_ffill(product_info, 'Sector(s)')
product_info['Primary Essential Function'] = fix_missing_ffill(product_info, 'Primary Essential Function')
#district['pp_total_raw'] = fix_missing_ffill(district, 'pp_total_raw')


In [69]:
product_info['primary_function_main'] = product_info['Primary Essential Function'].apply(lambda x: x.split(' - ')[0] if x == x else x)
product_info['primary_function_sub'] = product_info['Primary Essential Function'].apply(lambda x: x.split(' - ')[1] if x == x else x)

# Synchronize similar values
product_info['primary_function_sub'] = product_info['primary_function_sub'].replace({'Sites, Resources & References' : 'Sites, Resources & Reference'})
product_info.drop("Primary Essential Function", axis=1, inplace=True)

In [70]:
product_info

,LP ID,URL,Product Name,Provider/Company Name,Sector(s),primary_function_main,primary_function_sub
0,13117,https://www.splashmath.com,SplashLearn,StudyPad Inc.,PreK-12,LC,Digital Learning Platforms
1,66933,https://abcmouse.com,ABCmouse.com,"Age of Learning, Inc",PreK-12,LC,Digital Learning Platforms
2,50479,https://www.abcya.com,ABCya!,"ABCya.com, LLC",PreK-12,LC,"Sites, Resources & Reference"
3,92993,http://www.aleks.com/,ALEKS,McGraw-Hill PreK-12,PreK-12; Higher Ed,LC,Digital Learning Platforms
4,73104,https://www.achieve3000.com/,Achieve3000,Achieve3000,PreK-12,LC,Digital Learning Platforms
...,...,...,...,...,...,...,...
367,88065,https://dochub.com/,DocHub,DocHub,PreK-12; Higher Ed; Corporate,SDO,Other
368,37805,http://google.com/slides/about/,Google Slides,Google LLC,PreK-12; Higher Ed; Corporate,LC,Content Creation & Curation
369,32555,http://www.innersloth.com/gameAmongUs.php,Among Us,InnerSloth,PreK-12; Higher Ed,LC,"Sites, Resources & Reference"
370,87841,http://edpuzzle.com,Edpuzzle - Free (Basic Plan),EDpuzzle Inc.,PreK-12; Higher Ed,LC,"Sites, Resources & Reference"


In [71]:
product_info.isna().sum()

LP ID                    0
URL                      0
Product Name             0
Provider/Company Name    1
Sector(s)                0
primary_function_main    0
primary_function_sub     0
dtype: int64

In [72]:
product_info = product_info.dropna()

In [73]:
product_info.isna().sum()

LP ID                    0
URL                      0
Product Name             0
Provider/Company Name    0
Sector(s)                0
primary_function_main    0
primary_function_sub     0
dtype: int64

In [77]:
merged_data = pd.merge(product_info, engagement, left_on = 'LP ID', right_on = 'lp_id')
merged_data['district_id'] = merged_data['district_id'].astype('int64')
merged_data = pd.merge(merged_data, district, on = 'district_id')
#merged_data.drop(['URL', 'lp_id'], axis = 1, inplace = True)
merged_data.head(3)

,LP ID,URL,Product Name,Provider/Company Name,Sector(s),primary_function_main,primary_function_sub,time,lp_id,pct_access,engagement_index,district_id,holiday,outbreak,month,weekday,state,locale,pct_black/hispanic,pct_free/reduced,pp_total_raw
0,13117,https://www.splashmath.com,SplashLearn,StudyPad Inc.,PreK-12,LC,Digital Learning Platforms,2020-02-27,13117.0,0.14,2.84,8815,0,0,February,Thursday,Illinois,Suburb,0.1,0.1,15000
1,13117,https://www.splashmath.com,SplashLearn,StudyPad Inc.,PreK-12,LC,Digital Learning Platforms,2020-03-02,13117.0,0.18,6.13,8815,0,0,March,Monday,Illinois,Suburb,0.1,0.1,15000
2,13117,https://www.splashmath.com,SplashLearn,StudyPad Inc.,PreK-12,LC,Digital Learning Platforms,2020-03-03,13117.0,0.08,3.18,8815,0,0,March,Tuesday,Illinois,Suburb,0.1,0.1,15000


In [78]:
merged_data.drop(['URL', 'lp_id'], axis = 1, inplace = True)

In [79]:
merged_data.head(3)

,LP ID,Product Name,Provider/Company Name,Sector(s),primary_function_main,primary_function_sub,time,pct_access,engagement_index,district_id,holiday,outbreak,month,weekday,state,locale,pct_black/hispanic,pct_free/reduced,pp_total_raw
0,13117,SplashLearn,StudyPad Inc.,PreK-12,LC,Digital Learning Platforms,2020-02-27,0.14,2.84,8815,0,0,February,Thursday,Illinois,Suburb,0.1,0.1,15000
1,13117,SplashLearn,StudyPad Inc.,PreK-12,LC,Digital Learning Platforms,2020-03-02,0.18,6.13,8815,0,0,March,Monday,Illinois,Suburb,0.1,0.1,15000
2,13117,SplashLearn,StudyPad Inc.,PreK-12,LC,Digital Learning Platforms,2020-03-03,0.08,3.18,8815,0,0,March,Tuesday,Illinois,Suburb,0.1,0.1,15000


In [80]:
merged_data.shape

(9139701, 19)